In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
from splinter import Browser
import json
from sqlalchemy import create_engine
import numpy as np
import plotly.express as px


In [2]:
poverty_file='Resources/poverty.csv'
poverty_df=pd.read_csv(poverty_file)
poverty_df

,Year,State,County ID,State / County Name,All Ages SAIPE Poverty Universe,All Ages in Poverty Count,All Ages in Poverty Count LB 90%,All Ages in Poverty Count UB 90%,90% Confidence Interval (All Ages in Poverty Count),All Ages in Poverty Percent,...,Under Age 5 in Poverty Count UB 90%,90% Confidence Interval (Under Age 5 in Poverty Count),Under Age 5 in Poverty Percent,Under Age 5 in Poverty Percent LB 90%,Under Age 5 in Poverty Percent UB 90%,90% Confidence Interval (Under Age 5 in Poverty Percent),Median Household Income in Dollars,Median Household Income in Dollars LB 90%,Median Household Income in Dollars UB 90%,90% Confidence Interval (Median Household Income in Dollars)
0,2018,0,0,United States,"319,184,033","41,852,315","41,619,366","42,085,264","41,619,366 to 42,085,264",13.1,...,"3,802,546","3,714,862 to 3,802,546",19.5,19.3,19.7,19.3 to 19.7,"$61,937","$61,843","$62,031","$61,843 to $62,031"
1,2018,1,1000,Alabama,"4,763,811","801,758","785,668","817,848","785,668 to 817,848",16.8,...,"77,840","69,990 to 77,840",26.0,24.6,27.4,24.6 to 27.4,"$49,881","$49,123","$50,639","$49,123 to $50,639"
2,2018,1,1001,Autauga County (AL),"55,073","7,587","6,334","8,840","6,334 to 8,840",13.8,...,NaN,NaN,NaN,NaN,NaN,NaN,"$59,338","$53,628","$65,048","$53,628 to $65,048"
3,2018,1,1003,Baldwin County (AL),"215,255","21,069","17,390","24,748","17,390 to 24,748",9.8,...,NaN,NaN,NaN,NaN,NaN,NaN,"$57,588","$54,437","$60,739","$54,437 to $60,739"
4,2018,1,1005,Barbour County (AL),"21,979","6,788","5,662","7,914","5,662 to 7,914",30.9,...,NaN,NaN,NaN,NaN,NaN,NaN,"$34,382","$31,157","$37,607","$31,157 to $37,607"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,2018,56,56037,Sweetwater County (WY),"42,205","3,540","2,804","4,276","2,804 to 4,276",8.4,...,NaN,NaN,NaN,NaN,NaN,NaN,"$73,315","$67,141","$79,489","$67,141 to $79,489"
3190,2018,56,56039,Teton County (WY),"22,888","1,443","1,137","1,749","1,137 to 1,749",6.3,...,NaN,NaN,NaN,NaN,NaN,NaN,"$99,087","$88,142","$110,032","$88,142 to $110,032"
3191,2018,56,56041,Uinta County (WY),"20,135","2,010","1,585","2,435","1,585 to 2,435",10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"$63,401","$56,812","$69,990","$56,812 to $69,990"
3192,2018,56,56043,Washakie County (WY),"7,735",918,700,"1,136","700 to 1,136",11.9,...,NaN,NaN,NaN,NaN,NaN,NaN,"$55,190","$49,263","$61,117","$49,263 to $61,117"


In [3]:
narrowed_poverty_df=poverty_df[['Year', 'State', 'County ID', 'State / County Name','All Ages in Poverty Count', 'All Ages in Poverty Percent']]
narrowed_poverty_df

,Year,State,County ID,State / County Name,All Ages in Poverty Count,All Ages in Poverty Percent
0,2018,0,0,United States,"41,852,315",13.1
1,2018,1,1000,Alabama,"801,758",16.8
2,2018,1,1001,Autauga County (AL),"7,587",13.8
3,2018,1,1003,Baldwin County (AL),"21,069",9.8
4,2018,1,1005,Barbour County (AL),"6,788",30.9
...,...,...,...,...,...,...
3189,2018,56,56037,Sweetwater County (WY),"3,540",8.4
3190,2018,56,56039,Teton County (WY),"1,443",6.3
3191,2018,56,56041,Uinta County (WY),"2,010",10.0
3192,2018,56,56043,Washakie County (WY),918,11.9


In [4]:
html_table=narrowed_poverty_df.to_html()
html_table=html_table.replace('\n', '')


In [5]:
unemp_file="Resources/unemployment.csv"
unemp_df=pd.read_csv(unemp_file)
unemp_df.head()

,LAUS Code,State Code,County Code,County Name/State Abbreviation,Year,Labor Force,Employed,Unemployed,Unemployment Rate(%),County ID
0,CN0100100000000,1,1,"Autauga County, AL",2018,"26,196","25,261",935,3.6,1001
1,CN0100300000000,1,3,"Baldwin County, AL",2018,"95,233","91,809","3,424",3.6,1003
2,CN0100500000000,1,5,"Barbour County, AL",2018,"8,414","7,987",427,5.1,1005
3,CN0100700000000,1,7,"Bibb County, AL",2018,"8,605","8,268",337,3.9,1007
4,CN0100900000000,1,9,"Blount County, AL",2018,"25,069","24,201",868,3.5,1009


In [6]:
narrowed_unemp=unemp_df[['County ID','Labor Force','Employed','Unemployed','Unemployment Rate(%)']]
narrowed_unemp.head()

,County ID,Labor Force,Employed,Unemployed,Unemployment Rate(%)
0,1001,"26,196","25,261",935,3.6
1,1003,"95,233","91,809","3,424",3.6
2,1005,"8,414","7,987",427,5.1
3,1007,"8,605","8,268",337,3.9
4,1009,"25,069","24,201",868,3.5


In [7]:
combined_df=narrowed_poverty_df.merge(narrowed_unemp,how='outer', on='County ID')
combined_df.sample(50)

,Year,State,County ID,State / County Name,All Ages in Poverty Count,All Ages in Poverty Percent,Labor Force,Employed,Unemployed,Unemployment Rate(%)
2880,2018,51,51025,Brunswick County (VA),"3,187",22.5,"6,000","5,699",301,5.0
989,2018,20,20167,Russell County (KS),932,13.7,"3,358","3,261",97,2.9
1734,2018,31,31103,Keya Paha County (NE),148,18.3,593,580,13,2.2
1573,2018,29,29127,Marion County (MO),"3,686",13.5,"14,451","14,010",441,3.1
1601,2018,29,29183,St. Charles County (MO),"22,272",5.7,"224,590","219,001","5,589",2.5
130,2018,5,5029,Conway County (AR),"3,693",17.9,"8,174","7,821",353,4.3
2905,2018,51,51077,Grayson County (VA),"2,686",18.4,"7,667","7,430",237,3.1
2164,2018,39,39169,Wayne County (OH),"10,496",9.4,"61,098","58,905","2,193",3.6
1396,2018,27,27115,Pine County (MN),"3,319",12.0,"14,868","14,139",729,4.9
1136,2018,22,22007,Assumption Parish (LA),"4,562",20.7,"8,960","8,365",595,6.6


In [8]:
mo=combined_df[combined_df['State']==29]
mo=mo.reset_index(drop=True)
mo

,Year,State,County ID,State / County Name,All Ages in Poverty Count,All Ages in Poverty Percent,Labor Force,Employed,Unemployed,Unemployment Rate(%)
0,2018,29,29000,Missouri,"785,343",13.2,NaN,NaN,NaN,NaN
1,2018,29,29001,Adair County (MO),"5,356",23.9,"10,631","10,259",372,3.5
2,2018,29,29003,Andrew County (MO),"1,429",8.2,"9,725","9,472",253,2.6
3,2018,29,29005,Atchison County (MO),600,11.8,"2,768","2,700",68,2.5
4,2018,29,29007,Audrain County (MO),"3,883",16.7,"10,798","10,469",329,3.0
...,...,...,...,...,...,...,...,...,...,...
111,2018,29,29223,Wayne County (MO),"3,018",23.3,"5,341","5,088",253,4.7
112,2018,29,29225,Webster County (MO),"5,384",14.1,"16,987","16,467",520,3.1
113,2018,29,29227,Worth County (MO),279,14.1,"1,193","1,159",34,2.8
114,2018,29,29229,Wright County (MO),"4,330",23.9,"7,427","7,123",304,4.1


In [9]:
mo_data='mo_data.csv'
mo.to_csv(mo_data)

In [10]:

client = pymongo.MongoClient('localhost', 27017)
mng_db = client['UnemploymentDB'] 
collection_name = 'Missouri' 
db = mng_db[collection_name]

data = pd.read_csv(mo_data)
data_json = json.loads(data.to_json(orient='records'))
db.insert_many(data_json)

In [11]:
list(mng_db.Missouri.find())


[{'_id': ObjectId('5ec3272070823537a4619259'),
  'Unnamed: 0': 0,
  'Year': 2018,
  'State': 29,
  'County ID': 29000,
  'State / County Name': 'Missouri',
  'All Ages in Poverty Count': '785,343',
  'All Ages in Poverty Percent': 13.2,
  'Labor Force': None,
  'Employed': None,
  'Unemployed': None,
  'Unemployment Rate(%)': None},
 {'_id': ObjectId('5ec3272070823537a461925a'),
  'Unnamed: 0': 1,
  'Year': 2018,
  'State': 29,
  'County ID': 29001,
  'State / County Name': 'Adair County (MO)',
  'All Ages in Poverty Count': '5,356',
  'All Ages in Poverty Percent': 23.9,
  'Labor Force': '10,631     ',
  'Employed': '10,259     ',
  'Unemployed': '372     ',
  'Unemployment Rate(%)': 3.5},
 {'_id': ObjectId('5ec3272070823537a461925b'),
  'Unnamed: 0': 2,
  'Year': 2018,
  'State': 29,
  'County ID': 29003,
  'State / County Name': 'Andrew County (MO)',
  'All Ages in Poverty Count': '1,429',
  'All Ages in Poverty Percent': 8.2,
  'Labor Force': '9,725     ',
  'Employed': '9,472     

In [18]:
x=mo['All Ages in Poverty Percent']
x=x[1:]
y=mo['Unemployment Rate(%)']
y=y[1:]


In [24]:
fig = px.scatter(x=x, y=y, trendline='ols', labels={'x':'Poverty Percent', 'y':'Unemployment Rate(%)'})
fig.show()

In [31]:
x1=combined_df['All Ages in Poverty Percent']
y1=combined_df['Unemployment Rate(%)']


In [30]:
fig = px.scatter(x=x1, y=y1, trendline='ols', labels={'x':'Poverty Percent', 'y':'Unemployment Rate(%)'})
fig.show()